In [2]:
import sqlite3
import pyodbc
import pandas as pd
import numpy as np
from collections import defaultdict
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
DB = {
    'servername': 'tcp:dwhunitedoutdoors.database.windows.net,1433',
    'database': 'DWH_United_Outdoors',
    'username': 'onderzoeker',
    'password': 'Wachtwoord1.' 
}

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=' + DB['servername'] + ';'
    'DATABASE=' + DB['database'] + ';'
    'UID=' + DB['username'] + ';'
    'PWD=' + DB['password'] + ';'
    'Encrypt=yes;'
    'TrustServerCertificate=no;'
    'Connection Timeout=30;'
)

dwh_conn = pyodbc.connect(conn_str)
dwh_cursor = dwh_conn.cursor()
dwh_cursor 

merge va de bestelling tabellen 

In [ ]:
blank = df.groupby('blank')['blank'].apply(list).tolist()

te = TransactionEncoder()
te_ary = te.fit(blank).transform(blank)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# 4. Definieer een functie om het Apriori-algoritme uit te voeren en associatieregels te genereren
def find_frequent_itemsets(df, min_support, min_confidence):
    frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    return frequent_itemsets, rules

# 5. Voer een grid search uit om de beste parameters te vinden
best_support = 0
best_confidence = 0
best_score = 0
best_frequent_itemsets = None
best_rules = None

support_values = [0.5, 0.6, 0.7]
confidence_values = [0.6, 0.7, 0.8]

for support in support_values:
    for confidence in confidence_values:
        frequent_itemsets, rules = find_frequent_itemsets(df_encoded, support, confidence)
        score = len(rules)  # Voorbeeld evaluatiemetric: aantal regels
        if score > best_score:
            best_score = score
            best_support = support
            best_confidence = confidence
            best_frequent_itemsets = frequent_itemsets
            best_rules = rules

# 6. Toon de beste frequent itemsets en associatieregels
print(f"Beste minimale support: {best_support}")
print(f"Beste minimale confidence: {best_confidence}")
print(f"Aantal regels: {best_score}")

print("\nBeste Frequent Itemsets:")
print(best_frequent_itemsets)

print("\nBeste Association Rules:")
print(best_rules)